## 데이터 전처리

In [23]:
import pandas as pd

In [24]:
kospi50_daily_stock_chart = pd.read_csv("./data/kospi50-1980-2021.csv")
kospi50_daily_stock_chart

,code,name,date,open,high,low,close
0,A105560,KB금융,20081010,47063,47063,40026,45939
1,A105560,KB금융,20081013,47063,50338,45499,49556
2,A105560,KB금융,20081014,52781,54736,50631,51902
3,A105560,KB금융,20081015,49067,50142,48578,49849
4,A105560,KB금융,20081016,44620,44913,42420,42420
...,...,...,...,...,...,...,...
270830,A005380,현대차,20210830,211000,212500,210000,210000
270831,A005380,현대차,20210831,210000,212500,208000,212500
270832,A005380,현대차,20210901,214000,218500,213500,215000
270833,A005380,현대차,20210902,214000,215000,210000,211500


In [29]:
def preprocess(stock_chart):
    z_feature = stock_chart[["code", "name", "date"]].values.tolist()
    stock_chart = stock_chart.values.tolist()
    stock_first_index = 0
    
    for idx, daily in enumerate(stock_chart):
        z_feature[idx].append(daily[3] / daily[6] - 1)    # open
        z_feature[idx].append(daily[4] / daily[6] - 1)    # high
        z_feature[idx].append(daily[5] / daily[6] - 1)    # low
        
        if stock_chart[idx-1][1] != daily[1]:
            s_idx = idx
            for i in range(7):
                z_feature[idx].append(0) # close
            continue
        else:
            z_feature[idx].append(daily[6] / stock_chart[idx-1][6] - 1) # close
        
        for k in [5, 10, 15, 20, 25, 30]: # z_dk
            if (idx-s_idx)-(k-1) < 0:
                sum_close = stock_chart[s_idx:idx+1]
            else:
                sum_close = stock_chart[idx-(k-1):idx+1]

            sum_close = [chart[6] for chart in sum_close]
            sum_close = sum(sum_close) / (len(sum_close) * daily[6]) - 1
            z_feature[idx].append(sum_close)

    return pd.DataFrame(z_feature, columns=["code", "name", "date", "open", "high", "low", "close",
                                            "z_d5", "z_d10", "z_d15", "z_d20", "z_d25", "z_d30"])


In [30]:
preprocessed_kospi50_daily_stock_chart = preprocess(kospi50_daily_stock_chart)

In [31]:
preprocessed_kospi50_daily_stock_chart

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,A105560,KB금융,20081010,0.024467,0.024467,-0.128714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,A105560,KB금융,20081013,-0.050307,0.015780,-0.081867,0.078735,-0.036494,-0.036494,-0.036494,-0.036494,-0.036494,-0.036494
2,A105560,KB금융,20081014,0.016936,0.054603,-0.024488,0.047340,-0.053363,-0.053363,-0.053363,-0.053363,-0.053363,-0.053363
3,A105560,KB금융,20081015,-0.015687,0.005878,-0.025497,-0.039555,-0.010783,-0.010783,-0.010783,-0.010783,-0.010783,-0.010783
4,A105560,KB금융,20081016,0.051862,0.058769,0.000000,-0.149030,0.129967,0.129967,0.129967,0.129967,0.129967,0.129967
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270830,A005380,현대차,20210830,0.004762,0.011905,0.000000,0.007194,-0.002381,-0.007143,0.009365,0.022262,0.029619,0.038413
270831,A005380,현대차,20210831,-0.011765,0.000000,-0.021176,0.011905,-0.011294,-0.019059,-0.005176,0.008471,0.015247,0.023843
270832,A005380,현대차,20210901,-0.004651,0.016279,-0.006977,0.011765,-0.017674,-0.029302,-0.017984,-0.005233,0.001581,0.010078
270833,A005380,현대차,20210902,0.011820,0.016548,-0.007092,-0.016279,0.000000,-0.010875,-0.003940,0.008038,0.016076,0.024192


In [33]:
preprocessed_kospi50_daily_stock_chart[preprocessed_kospi50_daily_stock_chart.code == "A005490"]

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
54323,A005490,POSCO,19880610,0.038628,0.038628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
54324,A005490,POSCO,19880611,0.000000,0.000000,0.000000,-0.038656,0.020105,0.020105,0.020105,0.020105,0.020105,0.020105
54325,A005490,POSCO,19880613,0.000000,0.000000,0.000000,-0.032662,0.036366,0.036366,0.036366,0.036366,0.036366,0.036366
54326,A005490,POSCO,19880614,0.000000,0.000000,0.000000,-0.033765,0.054436,0.054436,0.054436,0.054436,0.054436,0.054436
54327,A005490,POSCO,19880615,0.000000,0.000000,0.000000,-0.034944,0.074093,0.074093,0.074093,0.074093,0.074093,0.074093
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63014,A005490,POSCO,20210830,-0.040663,0.003012,-0.043675,0.053968,-0.027410,-0.034940,-0.015361,-0.000602,0.017771,0.018273
63015,A005490,POSCO,20210831,-0.013353,0.000000,-0.026706,0.015060,-0.031751,-0.047033,-0.030069,-0.018843,0.000653,0.003412
63016,A005490,POSCO,20210901,-0.014815,0.014815,-0.014815,0.001484,-0.026370,-0.045926,-0.031506,-0.020815,-0.004030,0.002667
63017,A005490,POSCO,20210902,-0.013139,0.011679,-0.016058,0.014815,-0.028321,-0.051387,-0.046229,-0.035693,-0.021489,-0.010803


In [34]:
preprocessed_kospi50_daily_stock_chart.to_csv("./data/preprocessed_kospi50_daily_stock_chart.csv", index=False)